# Chapter 8

# Data Wrangling: Join, Combine, and Reshape

### Heirarchical Indexing

In [3]:
import sys
import numpy as np
import pandas as pd
import csv
import json
from lxml import objectify
import pyarrow
import openpyxl
import xlrd
from bs4 import BeautifulSoup
import sqlalchemy as sqla
import sqlite3
import os
from pathlib import Path

In [4]:
data = pd.Series(np.random.uniform(size=9), 
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"], [1,2,3,1,3,1,2,2,3]])

In [5]:
data

a  1    0.206212
   2    0.840512
   3    0.438275
b  1    0.606792
   3    0.003035
c  1    0.691266
   2    0.952425
d  2    0.483153
   3    0.846258
dtype: float64

In [6]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [7]:
data['b']

1    0.606792
3    0.003035
dtype: float64

In [8]:
data['b':'c']

b  1    0.606792
   3    0.003035
c  1    0.691266
   2    0.952425
dtype: float64

In [9]:
data.loc[["b", "d"]]

b  1    0.606792
   3    0.003035
d  2    0.483153
   3    0.846258
dtype: float64

In [10]:
data.loc[:,2]

a    0.840512
c    0.952425
d    0.483153
dtype: float64

In [11]:
data.unstack()

,1,2,3
a,0.206212,0.840512,0.438275
b,0.606792,NaN,0.003035
c,0.691266,0.952425,NaN
d,NaN,0.483153,0.846258


In [12]:
data.unstack().stack()

a  1    0.206212
   2    0.840512
   3    0.438275
b  1    0.606792
   3    0.003035
c  1    0.691266
   2    0.952425
d  2    0.483153
   3    0.846258
dtype: float64

In [14]:
frame = pd.DataFrame(np.arange(12).reshape((4,3,)),
                     index=[['a','a','b','b'], [1,2,1,2]],
                     columns=[["Ohio", "Ohio", "Colorado"], 
                              ["Green", "Red", "Green"]])

In [15]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [16]:
frame.index.names = ['key1', 'key2']

In [17]:
frame.columns.names= ["state", "color"]

In [18]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [19]:
frame.index.nlevels

2

In [20]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [21]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]], names = ["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

In [22]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [23]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [24]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics By Level

In [25]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [28]:
frame.groupby(level="color", axis="columns").sum()

/var/folders/fy/thq20n_j7rg3ycyq1v1s6gzh0000gp/T/ipykernel_2659/775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with DataFrame's Columns

In [30]:
frame = pd.DataFrame({"a": range(7), "b": range(7,0,-1), "c": ["one", "one", "one", "two", "two", "two", "two"], "d": [0,1,2,0,1,2,3]})

In [31]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [32]:
frame2= frame.set_index(["c", "d"])

In [33]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [34]:
frame.set_index(["c", "d"], drop = False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [35]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### Combining and Merging Datasets<br>
### Database Style DataFrame Joins

In [36]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

In [39]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [40]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [41]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [42]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [43]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [44]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], 
                   "data1": pd.Series(range(7), dtype="Int64")})

In [45]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                   "data2": pd.Series(range(3), dtype="Int64")})

Inner join is default:

In [46]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [47]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In an outer join the rows that don't match appear as NA values on the other DataFrame's columns

In [48]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how ="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


### Different join types with the how argument <br>
**how="inner"** <br> 
*Use only the key combinations observed in **both** tables.* <br><br>
**how="left"** <br> 
*Use all the key combinations found on left table.* <br><br>
**how="right"** <br> 
*Use all the key combinations found on right table.* <br><br>
**how="outer"** <br> 
*Use all the key combinations found on both tables.* <br><br>

In [49]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": pd.Series(range(6), dtype="Int64")})

In [50]:
df2= pd.DataFrame({"key": ["a", "b", "a", "b", "d"], 
                  "data2": pd.Series(range(5), dtype= "Int64")})

In [51]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [52]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [54]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [55]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [57]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                    "key2": ["one", "two", "one"],
                    "lval": pd.Series([1,2,3], dtype="Int64")})

In [59]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4,5,6,7], dtype='Int64')})

In [61]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [62]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [63]:
pd.merge(left,right, on = "key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Pandas Merge Function Arguments<br>

**left**<br>	*DataFrame to be merged on the left side.* <br><br>
**right**<br>	*DataFrame to be merged on the right side.* <br><br>
**how**<br>	*Type of join to apply: one of "inner", "outer", "left", or "right"; defaults to "inner".* <br><br>
**on**<br>	*Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys given, will use the intersection of the column names in left and right as the join keys.* <br><br>
**left_on**<br>	*Columns in left DataFrame to use as join keys. Can be a single column name or a list of column names.*<br><br>
**right_on**<br>	*Analogous to left_on for right DataFrame.*
**left_index**<br>	*Use row index in left as its join key (or keys, if a MultiIndex).* <br><br>
**right_index**<br>	*Analogous to left_index.* <br><br>
**sort**<br>	*Sort merged data lexicographically by join keys; False by default.* <br>
**suffixes**<br>	*Tuple of string values to append to column names in case of overlap; defaults to ("_x", "_y") (e.g., if "data" in both DataFrame objects, would appear as "data_x" and "data_y" in result).* <br><br>
**copy**<br>	*If False, avoid copying data into resulting data structure in some exceptional cases; by default always copies.* <br><br>
**validate** <br>	*Verifies if the merge is of the specified type, whether one-to-one, one-to-many, or many-to-many. See the docstring for full details on the options.* <br><br>
**indicator** <br>	*Adds a special column _merge that indicates the source of each row; values will be "left_only", "right_only", or "both" based on the origin of the joined data in each row.*

### Merging On Index

In [64]:
left1 = pd.DataFrame({'key': ["a", 'b', 'a', "a", 'b', 'c'],
                     "value": pd.Series(range(6), dtype="Int64")})

In [66]:
right1 = pd.DataFrame({"group_val": [3.5,7]}, index=["a", "b"])

In [67]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [68]:
right1

,group_val
a,3.5
b,7.0


In [70]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [71]:
pd.merge(left1, right1, left_on="key", right_index=True, how = "outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [72]:
lefth= pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
                    "key2": [2000, 2001, 2002, 2001, 2002],
                     "data": pd.Series(range(5), dtype="Int64")})

In [74]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada","Ohio","Ohio","Ohio","Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

In [77]:
righth = pd.DataFrame({"event1": pd.Series([0,2,4,6,8,10], dtype="Int64", 
                                          index= righth_index),
                       "event2": pd.Series([1,3,5,7,9,11], dtype= "Int64",
                                          index= righth_index)})

In [78]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [79]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [80]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [81]:
pd.merge(lefth, righth, left_on=["key1", "key2"], 
        right_index=True, how="outer")

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [86]:
left2= pd.DataFrame([[1.,2.], [3.,4.],[5.,6.]],
                   index=["a", "c", "e"],
                   columns=["Ohio", "Nevada"]).astype("Int64")

In [88]:
right2 = pd.DataFrame([[7.,8.], [9., 10.], [11.,12.],[13,14]],
                     index=["b", "c", "d", "e"],
                     columns=["Missouri", "Alabama"]).astype("Int64")

In [89]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [90]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [91]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [92]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [93]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [95]:
another = pd.DataFrame([[7.,8.], [9.,10.], [11.,12.], [16., 17.]],
                      index=['a', 'c', 'e', 'f'],
                      columns=['New York', 'Oregon'])

In [96]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [97]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [98]:
left2.join([right2, another], how = "outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


### Concatenating Along an Axis

In [99]:
arr = np.arange(12).reshape((3,4))

In [100]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [101]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [102]:
s1 = pd.Series([0,1], index = ["a", "b"], dtype="Int64")

In [103]:
s2 = pd.Series([2,3,4], index=["c","d","e"], dtype="Int64")

In [104]:
s3 = pd.Series([5,6], index= ["f", "g"], dtype="Int64")

In [105]:
s1

a    0
b    1
dtype: Int64

In [106]:
s2

c    2
d    3
e    4
dtype: Int64

In [107]:
s3

f    5
g    6
dtype: Int64

In [108]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [110]:
s4 = pd.concat([s1, s3])

In [111]:
pd.concat([s1,s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [112]:
pd.concat([s1,s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [113]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])

In [114]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [115]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [116]:
pd.concat([s1,s2,s3], axis= "columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [121]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=["a", "b", "c"],
                  columns=['one', 'two'])

In [122]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index=["a", "c"],
                  columns=['three', 'four'])

In [123]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [124]:
df2

,three,four
a,5,6
c,7,8


In [125]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [127]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [128]:
pd.concat([df1, df2], axis = "columns", keys=["level1", "level2"], names =['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [129]:
df1= pd.DataFrame(np.random.standard_normal((3,4)), columns=["b", "d", "a"])

ValueError: Shape of passed values is (3, 4), indices imply (3, 3)

In [130]:
df2= pd.DataFrame(np.random.standard_normal((2,3)), columns=["b", "d", "a"])

In [131]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [132]:
df2

,b,d,a
0,2.451607,-1.040820,1.029484
1,1.844402,-0.292123,-0.132664


In [133]:
pd.concat([df1, df2], ignore_index=True)

,one,two,b,d,a
0,0.0,1.0,NaN,NaN,NaN
1,2.0,3.0,NaN,NaN,NaN
2,4.0,5.0,NaN,NaN,NaN
3,NaN,NaN,2.451607,-1.040820,1.029484
4,NaN,NaN,1.844402,-0.292123,-0.132664


### Pandas.Concat Function Arguments <br> 

**objs**<br>	*List or dictionary of pandas objects to be concatenated; this is the only required argument*<br><br>
**axis**<br> *Axis to concatenate along; defaults to concatenating along rows (axis="index")* <br><br>
**join**<br>	*Either "inner" or "outer" ("outer" by default); whether to intersect (inner) or union (outer) indexes along the other axes*<br><br>
**keys**<br>	*Values to associate with objects being concatenated, forming a hierarchical index along the concatenation axis; can be a list or array of arbitrary values, an array of tuples, or a list of arrays (if multiple-level arrays passed in levels)*<br><br>
**levels**<br>	*Specific indexes to use as hierarchical index level or levels if keys passed*<br><br>
**names**<br>	*Names for created hierarchical levels if keys and/or levels passed*<br><br>
**verify_integrity**<br>	*Check new axis in concatenated object for duplicates and raise an exception if so; by default (False) allows duplicates*<br><br>
**ignore_index**<br>	*Do not preserve indexes along concatenation axis, instead produce a new range(total_length) index* 

### Combining Data with Overlap

In [134]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan], index=["f", "e", "d", "c", "b", "a"])

In [135]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
             index=["a", "b", "c", "d", "e", "f"])

In [136]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [137]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [138]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [139]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [140]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan], 
                    "b": [np.nan, 2., np.nan, 6],
                    "c": range(2,18,4)})

In [ ]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                   "b": [np.nan, 3., 4., 6., 8.]})